In [2]:
# load the package
library(scISR)

In [3]:
# load the pancreas dataset from ML4G_Project_2_Data/train_dat
data <- read.table("ML4G_Project_2_Data/train_data/pancreas_sc_train.csv", sep=",", header=TRUE, row.names=1)
bulk_data <- read.table("ML4G_Project_2_Data/train_data/pancreas_bulk_train.csv", sep=",", header=TRUE, row.names=1)

In [113]:
# log-transform the data
data <- log2(data+1)
bulk_data <- log2(bulk_data+1)

In [115]:
# apply imputation
imputed <- scISR(data, ncores=1, preprocessing = FALSE, batch_impute = TRUE)
write.csv(imputed, file="imputed_data_scISR_log.csv")


Using knn...

Clustering method: kmeans

Perturbation method: noise

Building original connectivity matrices



  |======================================================================| 100%

Building perturbed connectivity matrices



  |======================================================================| 100%


Done in 6.39689029455185 mins.




In [116]:
# compute mean of imputed data and bulk data for each gene
bulk_mean <- data.frame(mean=rowMeans(bulk_data))
imputed_mean <- data.frame(mean=rowMeans(imputed))

In [ ]:
# if preprocessing was applied, the rownames of imputed_mean and bulk_mean will be different

# get genes which are in the bulk data but not in the imputed data
only_in_bulk <- setdiff(rownames(bulk_mean), rownames(imputed_mean))

# create dataframe full of zeros with rownames only_in_bulk
zeros <- data.frame(mean = matrix(0, nrow=length(only_in_bulk), ncol=1),row.names = only_in_bulk)

# append dataframe rows of zeros to imputed_mean
imputed_mean_full <- rbind(imputed_mean, zeros)

# sort imputed mean by rownames
imputed_mean_full <- imputed_mean_full[order(rownames(imputed_mean_full)),]

# sort bulk mean by rownames
bulk_mean <- bulk_mean[order(rownames(bulk_mean)),]

In [117]:
# compute spearman correlation between two vectors
spearman <- function(x, y) {
  cor(x, y, method="spearman")
}

# compute the spearman correlation between the imputed and raw data
spearman(imputed_mean, bulk_mean)

,mean
mean,0.9463996


## 2. Clustering

In [11]:
# load libraries
library(irlba)
library(mclust)

Loading required package: Matrix

Package 'mclust' version 6.0.0
Type 'citation("mclust")' for citing this R package in publications.



In [ ]:
# load imputed data
imputed <- read.csv("ML4G_Project_2_Data/imputed_data/imputed_data_scISR_pp.csv", header=TRUE, row.names=1)

# load metadata
metadata <- read.csv("ML4G_Project_2_Data/train_data/pancreas_sc_metadata_train.csv", header=TRUE, row.names=1)